#### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Part 1:- Data Preprocessing

#### Preprocessing the Training Dataset

In [2]:
## To avoid overfitting in computer-vision we do pre-processing
## Well we apply some simple geometric transformations, rotations, zooms, flips. This is called Image Augmentation.

## Creating the object for the ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale = 1./255,        ## Feature Scaling which converts every pixels value between 0 and 1 just like Normalization
    shear_range = 0.2,       ## For rotations 
    zoom_range = 0.2,        ## For zooms 
    horizontal_flip = True   ## For flipping the images
)

In [3]:
training_set = train_datagen.flow_from_directory(
    'training_set',
    target_size = (150,150),   ## final size of the images when they will be fed to the CNN
    batch_size = 32,           ## no images in a batch
    class_mode = 'binary'      ## binary or categorical
)

Found 8000 images belonging to 2 classes.


#### Preprocessing the Test Dataset

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
test_set = test_datagen.flow_from_directory(
    'test_set',
    target_size =(150,150),
    batch_size = 32,
    class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.


### Part 2:- Building the CNN

#### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

#### Step-1: Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, input_shape = [150,150,3], activation = 'relu'))
## filters = no of filters
## kernel_size = n, n = nxn filter size
## input shape = (target_size,n), n=3 if RGB and n=1 if B/W. Only needed in the first convolution layer while connecting the input layer.
## No need to add input_shape in the second convolution layer.

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### Step-2: Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

#### Adding a second convoultion layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

#### Step-3: Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

#### Step-4: Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

#### Step-5: Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

### Part 3:- Training the CNN

#### Compiling the CNN

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#### Training the CNN on the Training Set and evaluating it on the Test Set

In [14]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 150)

Epoch 1/150


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 32s 123ms/step - accuracy: 0.5454 - loss: 0.7812 - val_accuracy: 0.5920 - val_loss: 0.6829
Epoch 2/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 113ms/step - accuracy: 0.6479 - loss: 0.6238 - val_accuracy: 0.7110 - val_loss: 0.5763
Epoch 3/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 113ms/step - accuracy: 0.6990 - loss: 0.5747 - val_accuracy: 0.6840 - val_loss: 0.5904
Epoch 4/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 114ms/step - accuracy: 0.7197 - loss: 0.5450 - val_accuracy: 0.7490 - val_loss: 0.5301
Epoch 5/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 115ms/step - accuracy: 0.7522 - loss: 0.5042 - val_accuracy: 0.7505 - val_loss: 0.5150
Epoch 6/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 113ms/step - accuracy: 0.7760 - loss: 0.4710 - val_accuracy: 0.7795 - val_loss: 0.4755
Epoch 7/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 30s 117ms/step - accuracy: 0.7832 - loss: 0.4584 - val_accuracy: 0.7610 - val_loss: 0.5032
Epoch 8/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 114ms/step - accuracy: 0.7927 - loss: 0.444

### Making a Single Prediction

In [45]:
## from keras.preprocessing import image
test_image = image.load_img('single_prediction/cat_or_dog_1.jpg', target_size = (150,150))

## Convert the image format(pil) to 2D array
test_image = image.img_to_array(test_image)

## As during training we took batch which contains number of images. So while prediction we also need to give the images in batch even if 
## it is a single image for which we are making prediction. So we add a fake dimension.
test_image = np.expand_dims(test_image, axis = 0)

result = cnn.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Dog'
else:
    prediction = 'Cat'
    
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Dog
